In [1]:
dotty.tools.dotc.config.Properties.simpleVersionString

res0: String = "3.2.2"

### 3.2 Pattern Matching

In [2]:
enum List[+A]:
    case Nil
    case Cons(head: A, tail: List[A])

object List:
    def apply[A](as: A*): List[A] =
        if as.isEmpty then Nil
        else Cons(as.head, apply(as.tail*))

    def sum(ints: List[Int]): Int = ints match
        case Nil => 0
        case Cons(x, xs) => x + sum(xs)

    def product(doubles: List[Double]): Double = doubles match
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x, xs) => x * product(xs)

defined class List
defined object List

#### Exercise 3.1

What is the result of the following match expression?

In [3]:
import List.*

import List.*


In [4]:
val result = List(1, 2, 3, 4, 5) match
    case Cons(x, Cons(2, Cons(4, _))) => x
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4, _)))) => x + y
    case Cons(h, t) => h + sum(t)
    case _ => 101

result: Int = 3

#### Exercise 3.2

Implement the function tail for removing the first element of a `List` (note that the function takes constasnt time). You can use `sys.error("message")` to throw an exception if the `List` is `Nil`. In the next chapter, we'll look at different ways of handling errors. Be careful to use the `List` enum and the `Nil` case defined here and not the built-in Scala `List` and `Nil` types.

In [5]:
def tail[A](xs: List[A]) = xs match
    case Nil => sys.error("Empty list doesn't have tail")
    case Cons(x, xs) => xs


defined function tail

In [6]:
tail(List(1, 2, 3))

res5: List[Int] = Cons(head = 2, tail = Cons(head = 3, tail = Nil))

In [7]:
tail(Nil)

: 

#### Exercise 3.3

Using the same idea, implement the function `setHead` for replacing the first element of a `List` with a different value.

In [8]:
def setHead[A](xs: List[A], head: A) = xs match
    case Nil => sys.error("Can't set head for an empty list")
    case Cons(x, xs) => Cons(head, xs)

defined function setHead

In [9]:
setHead(List(1, 2, 3), 4)

res8: List[Int] = Cons(
  head = 4,
  tail = Cons(head = 2, tail = Cons(head = 3, tail = Nil))
)

#### Exercise 3.4

Implement the function `drop`, which removes the first `n` elements from a list. Dropping `n` elements from an empty list should return the empty list. Note that this function takes time proportional only to the number of elements being dropped - we don't need to make a copy of the entire list

In [10]:
def drop[A](as: List[A], n: Int): List[A] = as match
    case Nil => Nil
    case Cons(x, xs) if n > 0 => drop(xs, n-1)
    case _ => as


defined function drop

In [11]:
drop(List(1, 2, 3, 4), 2)

res10: List[Int] = Cons(head = 3, tail = Cons(head = 4, tail = Nil))

In [12]:
drop(List(1, 2, 3, 4), 6)

res11: List[Int] = Nil

#### Exercise 3.5

Implement `dropWhile`, which removes elements from the `List` prefix as long as they match a predicate

In [13]:
def dropWhile[A](as: List[A], f: A => Boolean): List[A] = as match
    case Nil => Nil
    case Cons(x, xs) if f(x) => dropWhile(xs, f)
    case _ => as

defined function dropWhile

In [14]:
dropWhile(List(2, 4, 6, 7, 9, 10), _ % 2 == 0)

res13: List[Int] = Cons(
  head = 7,
  tail = Cons(head = 9, tail = Cons(head = 10, tail = Nil))
)

#### Exercise 3.6

Implement a function, `init`, that returns a `List` consisting of all but the last element of a `List`, so given `List(1, 2, 3, 4)`, `init` will return `List(1, 2, 3)`. Why can't this function be implemented in constant time (that is, runtime that's porportaionl to the size of the list) like tail?

In [15]:
def init[A](as: List[A]): List[A] = as match
    case Nil => sys.error("Can't take init of an empty list")
    case Cons(x, Nil) => Nil
    case Cons(x, xs) => Cons(x, init(xs))

defined function init

#### Exercise 3.7

Can `product`, implemented using `foldRight`, immediately hal the recursion and return `0.0` if it encounters a `0.0`? Why or why not? Consider how any short circuiting might work if you call `foldRight` with a large list. This is a deeper question, which we'll return to in chapter 5.

###### Answer
No, it's not possible for two reasons. First, the accumulator value that starts the product doesn't come into effect until we get to the end of the list. Second, there is no third case for short-circuiting in the `foldRight` function.


#### Exercise 3.8

See what happens when you pass `Nil` and `Cons` themselves to `foldRight`, like this: `foldRight(List(1, 2, 3), Nil: List[Int], Cons(_, _))`. What do you think this says about the relationship between `foldRight` and the data constructors of `List`?

In [16]:
def foldRight[A, B](as: List[A], acc: B, f: (A, B) => B): B = 
    as match
        case Nil => acc
        case Cons(x, xs) => f(x, foldRight(xs, acc, f))
        
foldRight(List(1, 2, 3), Nil: List[Int], Cons(_, _))

defined function foldRight
res15_1: List[Int] = Cons(
  head = 1,
  tail = Cons(head = 2, tail = Cons(head = 3, tail = Nil))
)

###### Answer

The result is the same and suggests that `foldRight` replaces `Nil` with `acc` and `f` with `Cons` in the list.

#### Exercise 3.9

Complete the length of a list using `foldRight`

In [18]:
def length[A](as: List[A]): Int = foldRight(as, 0, (_, len) => len + 1)

length(List(1, 2, 3))

defined function length
res17_1: Int = 3

#### Exercise 3.10

Our implementation of `foldRight` is not tail recursive and will result in a `StackoverflowError` for large lists (we say it's not *stack safe*). Convince yourself that this is the case, and then write another general list-recursion, `foldLeft`, that is tail recursive, using the techniques we discussed in the previous chapter. Start collapsing from the lefmost start of the list.

###### Answer

`foldLeft` is not tail recursive, because it calls `f` after making a recursive call, which means the last thing it does is not make a recursive call and therefore cannot be tail optimized.

In [20]:
def foldLeft[A, B](as: List[A], acc: B, f:(B, A) => B): B =
    as match
        case Nil => acc
        case Cons(x, xs) => foldLeft(xs, f(acc, x), f)

def sumLeft(as: List[Int]) = foldLeft(as, 0, _ + _)

sumLeft(List(1, 2, 3, 4))

defined function foldLeft
defined function sumLeft
res19_2: Int = 10

#### Exercise 3.11

Write `sum`, `product`, and a function to compute the length of a list using `foldLeft`.

In [21]:
def sum(as: List[Int]) = foldLeft(as, 0, _ + _)

def product(as: List[Double]) = foldLeft(as, 1.0, _ * _)

def length[T](as: List[T]) = foldLeft(as, 0, (acc, _) => acc + 1)

defined function sum
defined function product
defined function length

#### Exercise 3.12

Write a function that returns the reverse of a list (i.e., given `List(1, 2, 3)`, it returns `List(3, 2, 1)`). See if you can write it using a fold.

In [22]:
def reverse[T](as: List[T]): List[T] = foldLeft(as, Nil : List[T], (acc, a) => Cons(a, acc))

defined function reverse

#### Exercise 3.13

*Hard*: Can you write `foldRight` in terms of `foldLeft`? How about the other way around? Implementing `foldRight` via `foldLeft` is useful because it lets us implement `foldRight` tail recursively, which means it works even for large lists without overflowing the stack.

In [23]:
def foldRight[A, B](as: List[A], acc: B, f: (A, B) => B): B =
    foldLeft(reverse(as), acc, (acc, a) => f(a, acc))

defined function foldRight

#### Exercise 3.14

Recall the signature of `append`:

```scala
def append[A](a1: List[A], a2: List[A]): List[A]
```

Implement append in terms of either `foldLeft` or `foldRight` instead of structural recursion.

In [24]:
def append[A](a1: List[A], a2: List[A]): List[A] =
    foldRight(a1, a2, Cons(_, _))

defined function append

In [25]:
def append[A](a1: List[A], a2: List[A]): List[A] =
    foldLeft(reverse(a1), a2, (acc, a) => Cons(a, acc))

defined function append

#### Exercise 3.15

Write a function that concatenates a list of lists into a single list. Its runtime should be linear in the total length of all lists. Try to use functions we have already defined.

In [26]:
def flatten[A](xs: List[List[A]]): List[A] =
    foldRight(xs, Nil : List[A], append)

defined function flatten

#### Exercise 3.16

Write a function that transforms a list of integers by adding 1 to each element (that is, given a list of integer, it return a new list of integers where each value is one more than the corresponding value in the original list).

In [28]:
def inc(xs: List[Int]): List[Int] =
    foldRight(xs, Nil : List[Int], (x, acc) => Cons(x + 1, acc))

inc(List(1, 2, 3))

defined function inc
res27_1: List[Int] = Cons(
  head = 2,
  tail = Cons(head = 3, tail = Cons(head = 4, tail = Nil))
)

#### Exercise 3.17

Write a function that turns each value in a `List[Double]` into a `String`. You can use the expression `d.toString` to convert some `d: Double` to a `String`.

In [29]:
def dblsToStrings(ds: List[Double]): List[String] =
    foldRight(ds, Nil : List[String], (d, acc) => Cons(d.toString, acc))

defined function dblsToStrings

#### Exercise 3.18

Write a function `map`, that generalizes modifying each element in list while maintaining the structure of the list.

In [30]:
def map[A, B](as: List[A], f: A => B): List[B] =
    foldRight(as, Nil: List[B], (a, acc) => Cons(f(a), acc))

defined function map

Note: it's interesting that in Scala, which has eager evaluation, `map` can't be efficently implemented in a stack safe way functionally. It appears that it is actually implemented in imperatively in the standard library - https://github.com/scala/scala/blob/v2.13.6/src/library/scala/collection/immutable/List.scala#L244


#### Exercise 3.19

Write a function, `filter`, that removes elements from a list unless they satisfy a given predicate. Use it to remove all odd numbgers form a `List[Int]`.

In [31]:
def filter[A](as: List[A], f: A => Boolean): List[A] =
    foldRight(as, Nil : List[A], (a, acc) => if f(a) then Cons(a, acc) else acc)

defined function filter

#### Exercise 3.20

Write a function, `flatMap`, that works like `map` except that the function given will return a list instead of a single result, ensuring that the list is inserted into the final resulting list.

In [32]:
def flatMap[A, B](as: List[A], f: A => List[B]): List[B] =
    foldRight(as, Nil: List[B], (a, acc) => append(f(a), acc))

defined function flatMap

#### Exercise 3.21

Use `flatMap` to impelement `filter`.

In [34]:
def filter[A](as: List[A], f: A => Boolean): List[A] =
    flatMap(as, a => if f(a) then Cons(a, Nil) else Nil)

filter(List(1, 2, 3), _ > 1)

defined function filter
res33_1: List[Int] = Cons(head = 2, tail = Cons(head = 3, tail = Nil))

#### Exercise 3.22

Write a function that accepts two lists and constructs a new list by adding corresponding elements. For example, `List(1, 2, 3)` and `List(4, 5, 6)` becomes `List(5, 7, 9)`.

In [36]:
def addLists(a: List[Int], b: List[Int]): List[Int] =
    (a, b) match
        case (Nil, _) => Nil
        case (_, Nil) => Nil
        case (Cons(a, as), Cons(b, bs)) => Cons(a + b, addLists(as, bs))

addLists(List(1, 2, 3), List(4, 5, 6))

defined function addLists
res35_1: List[Int] = Cons(
  head = 5,
  tail = Cons(head = 7, tail = Cons(head = 9, tail = Nil))
)

### Exercise 3.23

Generalize the function you just wrote so it's not specific to integers or addition.

In [39]:
def zipWith[A, B, C](as: List[A], bs: List[B], f: (A, B) => C): List[C] =
    (as, bs) match
        case (Nil, _) => Nil
        case (_, Nil) => Nil
        case (Cons(a, as), Cons(b, bs)) => Cons(f(a, b), zipWith(as, bs, f))


def addLists(a: List[Int], b: List[Int]): List[Int] = zipWith(a, b, _ + _)
addLists(List(1, 2, 3), List(4, 5, 6))

defined function zipWith
defined function addLists
res38_2: List[Int] = Cons(
  head = 5,
  tail = Cons(head = 7, tail = Cons(head = 9, tail = Nil))
)

#### Exercise 3.24

*Hard*: Implement `hasSubsequence` to check whether a `List` contains another `List` as a subsequence. For instance, `List(1, 2, 3, 4)` would have `List(1, 2)` and `List(2, 3)` and `List(4)` as subsequences, among others. You may have some difficulty finding a concise a purely functional implementation that is also efficent.

In [83]:
def prefixesMatch[A](xs: List[A], ys: List[A]): Boolean =
    (xs, ys) match
        case (_, Nil) => true
        case (Nil, _) => false
        case (Cons(a, as), Cons(b, bs)) if a == b => prefixesMatch(as, bs)
        case _ => false

def hasSubsequence[A](sup: List[A], sub: List[A]): Boolean =
    (sup, sub) match
        case (_, Nil) => true
        case (Nil, _) => false
        case (Cons(a, as), Cons(b, bs)) if a == b => prefixesMatch(as, bs)
        case (Cons(a, as), _) => hasSubsequence(as, sub)

hasSubsequence(List(1, 2, 3, 4), List(1, 4))



defined function prefixesMatch
defined function hasSubsequence
res82_2: Boolean = false

#### Exercise 3.25

Write a function, `maximum`, that returns the maximum element in a Tree[Int]. (Note that in Scala you can use `x.max(y)` to compute the maximum of two integers `x` and `y`.)

In [84]:
enum Tree[+A]:
    case Leaf(value: A)
    case Branch(left: Tree[A], right: Tree[A])

    def size: Int = this match
        case Leaf(_) => 1
        case Branch(l, r) => 1 + l.size + r.size

def maximum(t: Tree[Int]): Int =
    t match
        case Tree.Leaf(a) => a
        case Tree.Branch(l, r) => maximum(l).max(maximum(r))

defined class Tree
defined function maximum

#### Exercise 3.26

Write a function, `depth`, that returns the maximum path length from the root of the tree to any leaf.

In [85]:
def depth[A](t: Tree[A]): Int =
    t match
        case Tree.Leaf(_) => 1
        case Tree.Branch(l, r) => 1 + depth(l).max(depth(r))

defined function depth

#### Exercise 3.27

Write a function, `map`, analogous to the method of the same name on `List` that modifies each element in a tree with a given function.

In [86]:
enum Tree[+A]:
    case Leaf(value: A)
    case Branch(left: Tree[A], right: Tree[A])

    def size: Int = this match
        case Leaf(_) => 1
        case Branch(l, r) => 1 + l.size + r.size

    def map[B](f: A => B): Tree[B] =
        this match
            case Leaf(a) => Leaf(f(a))
            case Branch(l, r) => Branch(l.map(f), r.map(f))

defined class Tree

#### Exercise 3.28

Generize `size`, `maximum`, `depth`, `map` write a new function, `fold`, that abstracts over their similarities. Reimplement them in terms of this more general function. Can you draw an analogy between this `fold` function and the left and right folds for `List`?

In [94]:
enum Tree[+A]:
    case Leaf(value: A)
    case Branch(left: Tree[A], right: Tree[A])

    def fold[B](f: A => B, merge: (B, B) => B): B =
        this match
            case Leaf(a) => f(a)
            case Branch(l, r) => merge(l.fold(f, merge), r.fold(f, merge))

    def size: Int = fold(_ => 1, _ + _ + 1)

    def depth: Int = fold(_ => 0, _ max _ + 1)

    def map[B](f: A => B): Tree[B] = fold(a => Leaf(f(a)), (l, r) => Branch(l, r))

import Tree.*

defined class Tree
import Tree.*


In [95]:
val tree = Branch(Branch(Leaf(1), Leaf(2)),
                  Branch(Leaf(3), Leaf(4)))

tree.size
tree.depth
tree.map(_ + 1)

tree: Tree[Int] = Branch(
  left = Branch(left = Leaf(value = 1), right = Leaf(value = 2)),
  right = Branch(left = Leaf(value = 3), right = Leaf(value = 4))
)
res94_1: Int = 7
res94_2: Int = 2
res94_3: Tree[Int] = Branch(
  left = Branch(left = Leaf(value = 2), right = Leaf(value = 3)),
  right = Branch(left = Leaf(value = 4), right = Leaf(value = 5))
)